# Detecting patterns of speciation in the fos- sil record

In this assignment, we use data from the NOW (New and Old Worlds) database of fossil mammals to study patterns of speciation over time and space. In particular, we are interested to know when and where speciation rates have been significantly high. The task is to find which time periods and which places over the history of mammals have given rise to exceptionally high numbers of new species. The phenomenon is known in the evolutionary literature as the “species factory”. Palaeontologists are interested why and in which ways those times and places are special. The role of computational science is to identify and characterize such times and places.
We practice using pandas DataFrames, performing logistic regression and making statistical significance tests in data analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

Exercise 2. Create a pandas DataFrame that contains all of the data
and save it as a csv file. How many rows does the DataFrame contain?

In [ ]:
df = pd.read_csv("now.txt", sep=',')

num_rows = len(df)
print(f"The DataFrame contains {num_rows} rows.")
df.to_csv("data.csv", index=False)

Exercise 3. a) Remove all rows where LAT = LONG = 0; these occurrences have incorrect coordinates. Drop rows where SPECIES is “sp.” or
“indet.”; these occurrences have not been properly identified.

In [ ]:
condition_lat_long_zero = (df['LAT'] == 0) & (df['LONG'] == 0)
df = df[~condition_lat_long_zero]
        
print(f"Number of rows after removing LAT=0 & LONG=0: {len(df)}")

species_to_remove = ["sp.", "indet."]
condition_unidentified_species = df['SPECIES'].isin(species_to_remove)
df = df[~condition_unidentified_species]

print(f"Number of rows after removing unidentified species ('sp.', 'indet.'): {len(df)}")

b) Next we will assign each occurrence to a specific Mammal Neogene
(MN) time unit. Table 1 shows the time boundaries of each time unit.
Assign each occurrence to a correct time unit by calculating the mean of
MIN AGE and MAX AGE. If the mean age of an occurrence is precisely
on the boundary between two time units, assign the occurrence to the
older time unit. If the mean age of an occurrence is outside of the MN
time interval, assign it to a “pre-MN” or “post-MN” category.

In [ ]:
mn_boundaries = {
    "MN1": (23.0, 21.7),
    "MN2": (21.7, 19.5),
    "MN3": (19.5, 17.2),
    "MN4": (17.2, 16.4),
    "MN5": (16.4, 14.2),
    "MN6": (14.2, 12.85),
    "MN7-8": (12.85, 11.2),
    "MN9": (11.2, 9.9),
    "MN10": (9.9, 8.9),
    "MN11": (8.9, 7.6),
    "MN12": (7.6, 7.1),
    "MN13": (7.1, 5.3),
    "MN14": (5.3, 5.0),
    "MN15": (5.0, 3.55),
    "MN16": (3.55, 2.5),
    "MN17": (2.5, 1.9),
    "MQ18": (1.9, 0.85), 
    "MQ19": (0.85, 0.01) 
}

overall_start_age = mn_boundaries["MN1"][0]
overall_end_age = mn_boundaries["MQ19"][1]

""" Calculate mean age, handling potential NaN values if necessary
    Assigns an occurrence to an MN unit based on its mean age."""
def assign_mn_unit(row):
    if pd.isna(row['MIN AGE']) or pd.isna(row['MAX AGE']):
        return 'unknown_age' 
    mean_age = (row['MIN AGE'] + row['MAX AGE']) / 2.0
    if mean_age > overall_start_age:
        return 'pre-MN'
    elif mean_age < overall_end_age: 
        return 'post-MN'
    else:
        for unit, (start_age, end_age) in mn_boundaries.items():
            if mean_age >= end_age and mean_age < start_age:
                return unit
        if mean_age == overall_end_age:
             for unit, (start_age, end_age) in mn_boundaries.items():
                 if end_age == overall_end_age:
                     return unit
    return 'assignment_error' 
df['MIN AGE'] = pd.to_numeric(df['MIN AGE'], errors='coerce')
df['MAX AGE'] = pd.to_numeric(df['MAX AGE'], errors='coerce')
df['MN_Unit'] = df.apply(assign_mn_unit, axis=1)
print("First 5 rows with assigned MN Unit:")
print(df[['LIDNUM', 'MIN AGE', 'MAX AGE', 'MN_Unit']].head())
print("\nCounts per MN Unit:")
print(df['MN_Unit'].value_counts().sort_index())
print("\nChecking for assignment issues:")
print(df[df['MN_Unit'] == 'unknown_age'][['LIDNUM', 'MIN AGE', 'MAX AGE']].head())
print(df[df['MN_Unit'] == 'assignment_error'][['LIDNUM', 'MIN AGE', 'MAX AGE']].head())


c) Sometimes expert knowledge may be used to override some of the
information recorded in the data. In our case, experts in palaeontology
tell us that occurrences in the localities “Samos Main Bone Beds” and
“Can Llobateres I” should be assigned to time units MN12 and MN9,
respectively. Check these and if necessary, edit the time units to their
correct values.


In [ ]:
# Check if the specified localities exist in the dataset
samos = df[df['NAME'] == 'Samos Main Bone Beds']
can_llobateres = df[df['NAME'] == 'Can Llobateres I']

# Print current information
print("Before correction:")
print(f"Samos Main Bone Beds: {samos['MN_Unit'].unique() if len(samos) > 0 else 'Not found'}")
print(f"Can Llobateres I: {can_llobateres['MN_Unit'].unique() if len(can_llobateres) > 0 else 'Not found'}")

# Update MN units based on expert knowledge
df.loc[df['NAME'] == 'Samos Main Bone Beds', 'MN_Unit'] = 'MN12'
df.loc[df['NAME'] == 'Can Llobateres I', 'MN_Unit'] = 'MN9'

# Verify the changes
samos = df[df['NAME'] == 'Samos Main Bone Beds']
can_llobateres = df[df['NAME'] == 'Can Llobateres I']

print("\nAfter correction:")
print(f"Samos Main Bone Beds: {samos['MN_Unit'].unique() if len(samos) > 0 else 'Not found'}")
print(f"Can Llobateres I: {can_llobateres['MN_Unit'].unique() if len(can_llobateres) > 0 else 'Not found'}")
print(f"Number of records updated for Samos Main Bone Beds: {len(samos)}")
print(f"Number of records updated for Can Llobateres I: {len(can_llobateres)}")

d) We need to be able to identify all occurrences of each species. Assign a unique identification number for each unique combination of GENUS and SPECIES. Create a new column in the DataFrame and label each
occurrence with a corresponding species identification number.

In [ ]:
# Create a unique species identifier for each GENUS-SPECIES combination
# First create a concatenated field
df['full_species'] = df['GENUS'] + ' ' + df['SPECIES']

# Get unique combinations and create a mapping dictionary
unique_species = df['full_species'].unique()
species_id_map = {species: idx for idx, species in enumerate(unique_species, 1)}

# Assign IDs to each occurrence
df['species_id'] = df['full_species'].map(species_id_map)

# Display information about the species IDs
print(f"Total number of unique species: {len(unique_species)}")
print("\nFirst 10 species with their IDs:")
print(df[['GENUS', 'SPECIES', 'full_species', 'species_id']].head(10))

e) Each locality should contain no more than one occurrence of any
species. Check whether this is the case and remove duplicate copies, if
necessary.

In [ ]:
# Check for duplicates (same species in same locality)
duplicate_check = df.groupby(['LIDNUM', 'species_id']).size().reset_index(name='count')
duplicates = duplicate_check[duplicate_check['count'] > 1]

# Print information about duplicates
print(f"Number of locality-species combinations with duplicates: {len(duplicates)}")

if len(duplicates) > 0:
    print("\nExample of duplicates:")
    print(duplicates.head())
    
    # Get an example of duplicate records
    if not duplicates.empty:
        first_duplicate = duplicates.iloc[0]
        lid = first_duplicate['LIDNUM']
        sp_id = first_duplicate['species_id']
        
        print(f"\nDetailed view of a duplicate example (LIDNUM={lid}, species_id={sp_id}):")
        print(df[(df['LIDNUM'] == lid) & (df['species_id'] == sp_id)].head())
    
    # Remove duplicates - keep only the first occurrence of each species in each locality
    original_size = len(df)
    df = df.drop_duplicates(subset=['LIDNUM', 'species_id'])
    
    print(f"\nOriginal DataFrame size: {original_size}")
    print(f"After removing duplicates: {len(df)}")
    print(f"Number of duplicates removed: {original_size - len(df)}")
else:
    print("No duplicates found. Each species appears only once per locality.")

f) How many rows are we left with in the DataFrame (compare with
exercise 2)? How many unique species and localities are identified?

In [ ]:
# Current number of rows
current_rows = len(df)
print(f"Current number of rows in DataFrame: {current_rows}")

# Original number of rows from Exercise 2
print(f"Original number of rows (from Exercise 2): {num_rows}")
print(f"Difference: {num_rows - current_rows} rows were removed")

# Count unique species
unique_species_count = df['species_id'].nunique()
print(f"Number of unique species: {unique_species_count}")

# Count unique localities
unique_localities_count = df['LIDNUM'].nunique()
print(f"Number of unique localities: {unique_localities_count}")

Exercise 4. Create a DataFrame that shows for each species how many
occurrences it has in each time unit. Then, create a different DataFrame
that shows for each species the time unit when it is first observed (i.e.
the oldest time unit). For each time unit, calculate the proportion of first
occurrences to all occurrences. Plot the proportion of first occurrences
over time. Also, plot the total number of occurrences over time.

In [ ]:
# Step 1: DataFrame showing occurrences per species per time unit
species_time_occurrences = pd.crosstab(df['species_id'], df['MN_Unit'])

# Step 2: Find the first (oldest) time unit for each species
# Create a dictionary to map time units to their chronological order
mn_order = {
    "pre-MN": 0,
    "MN1": 1, "MN2": 2, "MN3": 3, "MN4": 4, "MN5": 5,
    "MN6": 6, "MN7-8": 7, "MN9": 8, "MN10": 9, "MN11": 10,
    "MN12": 11, "MN13": 12, "MN14": 13, "MN15": 14,
    "MN16": 15, "MN17": 16, "MQ18": 17, "MQ19": 18,
    "post-MN": 19
}

# Add chronological order column to assist in finding first occurrence
df['MN_Order'] = df['MN_Unit'].map(mn_order)

# Find first occurrence time unit for each species
first_occurrences = df.loc[df.groupby('species_id')['MN_Order'].idxmin()]
first_occ_df = pd.DataFrame({
    'species_id': first_occurrences['species_id'],
    'first_MN_Unit': first_occurrences['MN_Unit']
})

# Step 3: Calculate counts and proportions per time unit
total_occurrences = df['MN_Unit'].value_counts().sort_index()
first_occ_counts = first_occurrences['MN_Unit'].value_counts().sort_index()

# Create a DataFrame for the results
results = pd.DataFrame({
    'total_occurrences': total_occurrences,
    'first_occurrences': first_occ_counts.reindex(total_occurrences.index, fill_value=0)
})

# Calculate proportions
results['proportion'] = results['first_occurrences'] / results['total_occurrences']

# Step 4 & 5: Plot results
import matplotlib.pyplot as plt

# Filter out pre-MN and post-MN for better visualization
plot_results = results[~results.index.isin(['pre-MN', 'post-MN', 'unknown_age', 'assignment_error'])]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Proportion of first occurrences
ax1.bar(plot_results.index, plot_results['proportion'])
ax1.set_xlabel('Time Unit')
ax1.set_ylabel('Proportion of First Occurrences')
ax1.set_title('Proportion of First Occurrences by Time Unit')
ax1.set_ylim(0, 1)
ax1.grid(axis='y', linestyle='--', alpha=0.7)
plt.setp(ax1.get_xticklabels(), rotation=45)

# Plot 2: Total occurrences
ax2.bar(plot_results.index, plot_results['total_occurrences'])
ax2.set_xlabel('Time Unit')
ax2.set_ylabel('Number of Occurrences')
ax2.set_title('Total Occurrences by Time Unit')
ax2.grid(axis='y', linestyle='--', alpha=0.7)
plt.setp(ax2.get_xticklabels(), rotation=45)

plt.tight_layout()
plt.show()

# Print summary of the results
print("Summary of occurrences and first appearances by time unit:")
print(results[['total_occurrences', 'first_occurrences', 'proportion']])

Exercise 5. a) Create a DataFrame that collects the following information for every locality: locality number (LIDNUM), longitude, latitude,
time unit, number of first occurrences in the locality, number of all occurrences in the locality and proportion of first occurrences in the locality. Note, you should use LIDNUM to identify unique localities and not the NAME variable (why?).

In [ ]:
# Create a DataFrame with the required locality information
# 1. Get locality information (lat, long, time unit)
locality_info = df.groupby('LIDNUM').first().reset_index()
locality_df = locality_info[['LIDNUM', 'LONG', 'LAT', 'MN_Unit']]

# 2. Count all occurrences per locality
all_occurrences = df.groupby('LIDNUM').size().reset_index(name='total_occurrences')

# 3. Count first occurrences per locality
# Get species that are first occurrences
species_first_occ = df.loc[df.groupby('species_id')['MN_Order'].idxmin(), ['species_id', 'LIDNUM']]
first_occ_count = species_first_occ.groupby('LIDNUM').size().reset_index(name='first_occurrences')

# 4. Merge the information
locality_df = locality_df.merge(all_occurrences, on='LIDNUM', how='left')
locality_df = locality_df.merge(first_occ_count, on='LIDNUM', how='left')

# 5. Fill NAs with 0 (localities with no first occurrences)
locality_df['first_occurrences'] = locality_df['first_occurrences'].fillna(0)

# 6. Calculate proportion of first occurrences
locality_df['proportion_first'] = locality_df['first_occurrences'] / locality_df['total_occurrences']

# Display the results
print(f"Total number of localities: {len(locality_df)}")
print("\nFirst 5 localities:")
print(locality_df.head())

# Answer to why use LIDNUM instead of NAME:
print("\nWhy use LIDNUM instead of NAME?")
print("- LIDNUM is a unique identifier for each locality")
print("- NAME might not be unique (different localities could have the same name)")
print("- NAME could have variations in spelling, formatting, or data entry errors")
print("- Multiple localities might be part of the same named location but have distinct coordinates")

b) Visualize the distribution of localities in space and time. For each time
unit, plot the LAT and LONG coordinates of each locality (corresponding
to the time unit). For example, you can use the above codes to create a
geographic map and then use a standard matplotlib scatter plot to add
the localities. Choose the marker size for each locality such that it is
relative to the number of occurrences in the locality (bigger markers for
bigger localities).

In [ ]:
# Filter to only include standard MN time units
valid_mn_units = [
    "MN1", "MN2", "MN3", "MN4", "MN5", "MN6", "MN7-8", "MN9", "MN10", 
    "MN11", "MN12", "MN13", "MN14", "MN15", "MN16", "MN17", "MQ18", "MQ19"
]

# Filter out localities with invalid MN_Units
filtered_localities = locality_df[locality_df['MN_Unit'].isin(valid_mn_units)].copy()

# Set up the grid of plots
n_cols = 4  # Number of columns in the plot grid
n_rows = (len(valid_mn_units) + n_cols - 1) // n_cols  # Calculate number of rows needed

fig = plt.figure(figsize=(20, 5 * n_rows))

# Get min and max occurrences for consistent scaling across all plots
overall_min_occurrences = filtered_localities['total_occurrences'].min()
overall_max_occurrences = filtered_localities['total_occurrences'].max()

# Create a common normalization for all plots (for consistent colors)
norm = Normalize(vmin=overall_min_occurrences, vmax=overall_max_occurrences)
cmap = plt.cm.viridis

# Plot each time unit
for i, unit in enumerate(valid_mn_units):
    # Get data for this time unit
    unit_data = filtered_localities[filtered_localities['MN_Unit'] == unit]
    
    if len(unit_data) > 0:  # Only create plot if there are localities for this time unit
        ax = fig.add_subplot(n_rows, n_cols, i + 1)
        
        # Scale marker sizes based on number of occurrences (min 20, max 200)
        sizes = 20 + 180 * (unit_data['total_occurrences'] - overall_min_occurrences) / (overall_max_occurrences - overall_min_occurrences)
        sizes = np.maximum(sizes, 20)  # Ensure minimum size
        
        # Plot localities as scatter points
        scatter = ax.scatter(
            unit_data['LONG'], 
            unit_data['LAT'], 
            s=sizes,  # Size based on number of occurrences
            c=unit_data['total_occurrences'],  # Color based on number of occurrences
            cmap=cmap, 
            norm=norm,
            alpha=0.7, 
            edgecolor='black'
        )
        
        # Set plot limits to focus on the Europe-Asia-Africa area
        ax.set_xlim(-20, 60)
        ax.set_ylim(0, 60)
        
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        ax.set_title(f"{unit} ({len(unit_data)} localities)")
        ax.grid(True, linestyle='--', alpha=0.6)

# Add a colorbar to show the scale
cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])  # [left, bottom, width, height]
sm = ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('Number of Occurrences')

plt.tight_layout(rect=[0, 0, 0.9, 1])  # Adjust layout to make room for colorbar
plt.suptitle('Distribution of Fossil Mammal Localities by Time Unit', fontsize=16, y=0.92)
plt.show()

print("The visualization shows how fossil mammal localities are distributed across space for each time unit.")
print(f"Total number of localities plotted: {len(filtered_localities)}")
print(f"Range of occurrences per locality: {overall_min_occurrences} to {overall_max_occurrences}")

c) Based on exercises 4 and 5, what kind of observations about sampling
can you make? Are there differences in sampling density over space and
time? Compare some basic sampling properties between Africa, Asia and
Europe, e.g. spatial coverage and average number of occurrences per
locality

In [ ]:
# Define continental boundaries (approximate)
def assign_continent(row):
    # Europe: roughly -10 to 30 longitude, 35 to 70 latitude
    if -10 <= row['LONG'] <= 30 and 35 <= row['LAT'] <= 70:
        return 'Europe'
    # Africa: roughly -20 to 50 longitude, -40 to 35 latitude
    elif -20 <= row['LONG'] <= 50 and row['LAT'] < 35:
        return 'Africa'
    # Asia: roughly 30 to 150 longitude, 0 to 70 latitude
    elif row['LONG'] > 30 and 0 <= row['LAT'] <= 70:
        return 'Asia'
    else:
        return 'Other'

# Add continent information to the locality DataFrame
locality_df['continent'] = locality_df.apply(assign_continent, axis=1)

# Analyze sampling patterns by continent
continent_counts = locality_df['continent'].value_counts()
print("Number of localities by continent:")
print(continent_counts)

# Calculate average occurrences per locality by continent
avg_occurrences = locality_df.groupby('continent')['total_occurrences'].mean()
print("\nAverage occurrences per locality by continent:")
print(avg_occurrences)

# Get spatial coverage statistics
spatial_coverage = locality_df.groupby('continent').agg({
    'LAT': ['std', 'min', 'max'],
    'LONG': ['std', 'min', 'max'],
    'total_occurrences': ['sum', 'mean', 'median']
})
print("\nSpatial coverage and occurrence statistics by continent:")
print(spatial_coverage)

# Time distribution analysis
time_continent = pd.crosstab(locality_df['MN_Unit'], locality_df['continent'])
print("\nNumber of localities by time unit and continent:")
print(time_continent)

# Visualize sampling density over time by continent
valid_time_units = [u for u in valid_mn_units if u in time_continent.index]
plt.figure(figsize=(12, 6))
for continent in ['Europe', 'Asia', 'Africa']:
    if continent in time_continent.columns:
        plt.plot(valid_time_units, 
                 time_continent.loc[valid_time_units, continent], 
                 marker='o', 
                 label=continent)
plt.xlabel('Time Unit')
plt.ylabel('Number of Localities')
plt.title('Sampling Density Over Time by Continent')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Exercise 6. For each locality, look at a ten by ten degrees area (in
latitude and longitude) centered around the locality. Record the total
number of occurrences and total number of first occurrences found within
that square in the time unit corresponding to the focal locality. Also,
record the total number of occurrences within that square in the preceding
time unit (relative to the focal locality). Record these numbers into the
DataFrame that was created in exercise 5 (add new columns).

In [ ]:
# Define mapping to identify the preceding time unit
mn_order_list = [
    "pre-MN", "MN1", "MN2", "MN3", "MN4", "MN5", "MN6", "MN7-8", 
    "MN9", "MN10", "MN11", "MN12", "MN13", "MN14", "MN15", 
    "MN16", "MN17", "MQ18", "MQ19", "post-MN"
]

# Create a dictionary that maps each time unit to its preceding time unit
preceding_time_unit = {mn_order_list[i]: mn_order_list[i-1] for i in range(1, len(mn_order_list))}
preceding_time_unit["pre-MN"] = None  # No preceding time unit for pre-MN

# Initialize new columns in locality_df
locality_df['square_total_occurrences'] = 0
locality_df['square_first_occurrences'] = 0
locality_df['preceding_square_total_occurrences'] = 0

# For each locality, analyze the 10x10 degree area
for idx, locality in locality_df.iterrows():
    # Define the boundaries of the 10x10 degree square centered at the locality
    min_lat = locality['LAT'] - 5
    max_lat = locality['LAT'] + 5
    min_long = locality['LONG'] - 5
    max_long = locality['LONG'] + 5
    
    # Current time unit data
    current_time = locality['MN_Unit']
    
    # Find all localities in that square with the same time unit
    square_mask = (
        (locality_df['LAT'] >= min_lat) & 
        (locality_df['LAT'] <= max_lat) & 
        (locality_df['LONG'] >= min_long) & 
        (locality_df['LONG'] <= max_long) & 
        (locality_df['MN_Unit'] == current_time)
    )
    square_localities = locality_df[square_mask]
    
    # Count total occurrences in the square for the current time unit
    locality_df.at[idx, 'square_total_occurrences'] = square_localities['total_occurrences'].sum()
    
    # Count first occurrences in the square for the current time unit
    locality_df.at[idx, 'square_first_occurrences'] = square_localities['first_occurrences'].sum()
    
    # Get preceding time unit
    preceding_time = preceding_time_unit.get(current_time)
    
    if preceding_time:
        # Find all localities in that square from the preceding time unit
        preceding_square_mask = (
            (locality_df['LAT'] >= min_lat) & 
            (locality_df['LAT'] <= max_lat) & 
            (locality_df['LONG'] >= min_long) & 
            (locality_df['LONG'] <= max_long) & 
            (locality_df['MN_Unit'] == preceding_time)
        )
        preceding_square_localities = locality_df[preceding_square_mask]
        
        # Count total occurrences in the square for the preceding time unit
        locality_df.at[idx, 'preceding_square_total_occurrences'] = preceding_square_localities['total_occurrences'].sum()

# Display the results
print("Locality DataFrame with new square analysis columns:")
print(locality_df[['LIDNUM', 'LAT', 'LONG', 'MN_Unit', 
                   'total_occurrences', 'first_occurrences',
                   'square_total_occurrences', 'square_first_occurrences',
                   'preceding_square_total_occurrences']].head(10))

# Summary statistics
print("\nSummary statistics for the new columns:")
print(locality_df[['square_total_occurrences', 'square_first_occurrences', 
                  'preceding_square_total_occurrences']].describe())

Exercise 7. a) Create the regression data set. Only use localities within
the co-ordinates -25<LONG<40 and LAT>35 and time unit within MN2-
MQ19 (why not include MN1?). Create an m × 2 array, where m is the
total number of occurrences in all the localities. Each row in the array
represents one occurrence. For each occurrence, fill in to the first column
of the array the number of occurrences in the focal area in the previous
time unit (calculated in exercise 6). For the second column, fill in 1 for a
first occurrence and 0 for other occurrences.
b) Perform logistic regression.
c) Plot regression curve and 95%-confidence intervals.

In [ ]:
# Part a: Create the regression dataset
# Filter localities based on geographic and temporal constraints
filtered_localities = locality_df[
    (locality_df['LONG'] > -25) & (locality_df['LONG'] < 40) &
    (locality_df['LAT'] > 35) &
    (locality_df['MN_Unit'].isin(mn_order_list[2:19]))  # MN2-MQ19
].copy()

print(f"Number of localities after filtering: {len(filtered_localities)}")

# Create regression dataset - one row per occurrence
regression_data = []

for _, locality in filtered_localities.iterrows():
    # Get the total number of occurrences and first occurrences for this locality
    total_occurrences = int(locality['total_occurrences'])
    first_occurrences = int(locality['first_occurrences'])
    prev_occurrences = locality['preceding_square_total_occurrences']
    
    # Add rows for first occurrences (label = 1)
    for _ in range(first_occurrences):
        regression_data.append([prev_occurrences, 1])
    
    # Add rows for other occurrences (label = 0)
    for _ in range(total_occurrences - first_occurrences):
        regression_data.append([prev_occurrences, 0])

# Convert to numpy array
regression_array = np.array(regression_data)

print(f"Total occurrences in regression dataset: {len(regression_array)}")
print(f"Number of first occurrences: {sum(regression_array[:, 1])}")
print(f"First few rows of regression data:")
print(regression_array[:5])

# Part b: Perform logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Reshape X to be a 2D array
X = regression_array[:, 0].reshape(-1, 1)
y = regression_array[:, 1]

# Fit the model
model = LogisticRegression(random_state=42)
model.fit(X, y)

# Print model coefficients
print("\nLogistic Regression Results:")
print(f"Intercept: {model.intercept_[0]:.4f}")
print(f"Coefficient: {model.coef_[0][0]:.4f}")

# Part c: Plot regression curve with 95% confidence intervals
from scipy import stats

# Create a range of values for plotting
x_range = np.linspace(0, max(X) * 1.1, 1000).reshape(-1, 1)

# Predict probabilities
y_pred = model.predict_proba(x_range)[:, 1]

# Calculate standard error for confidence intervals
# Using the formula for the standard error of the log odds
n_samples = len(X)
p = y_pred
se = np.sqrt(p * (1 - p) / n_samples)

# Calculate confidence intervals (on the probability scale)
z = stats.norm.ppf(0.975)  # 95% CI
lower_ci = p - z * se
upper_ci = p + z * se

# Ensure bounds are between 0 and 1
lower_ci = np.maximum(0, lower_ci)
upper_ci = np.minimum(1, upper_ci)

# Plot
plt.figure(figsize=(10, 6))

# Plot the confidence interval as a shaded region
plt.fill_between(x_range.flatten(), lower_ci, upper_ci, color='gray', alpha=0.3, label='95% CI')

# Plot the regression curve
plt.plot(x_range, y_pred, 'r-', label='Predicted probability')

# Add scatter plot of the data
plt.scatter(X, y, color='blue', alpha=0.1, label='Data points')

# Add a title and labels
plt.title('Logistic Regression: Probability of First Occurrence vs. Previous Time Unit Occurrences')
plt.xlabel('Number of Occurrences in Previous Time Unit')
plt.ylabel('Probability of First Occurrence')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()

# Print model performance metrics
y_pred_class = model.predict(X)
print("\nModel Performance:")
print(classification_report(y, y_pred_class))

# Why not include MN1?
print("\nWhy not include MN1?")
print("MN1 is likely excluded because it's the earliest time unit in the series.")
print("Since we're looking at occurrences in the 'previous time unit', MN1 would have")
print("no previous time unit in this dataset (pre-MN might be too distant or sparse).")
print("This would create issues with the regression analysis as those data points")
print("would have missing or unreliable values for the predictor variable.")

Exercise 8. For each European locality, calculate the expected proportion of first occurrences in the focal area surrounding the locality using
the logistic regression calculated in exercise 7.

Exercise 9. For each European locality, calculate the probability of observing as many or more first occurrences in the focal area than what is
actually found. Assume that occurrences are binomially distributed to
“first occurrences” and “not first occurrences”, so that the probability of
a given occurrence to be a first occurrence is equal to the expected proportion of first occurrences in the focal area. You may use, for example,
the scipy.stats.binom library (https://docs.scipy.org/doc/scipy-0.14.0/
reference/generated/scipy.stats.binom.html) for the calculations.

In [ ]:
# Filter for European localities
european_localities = locality_df[locality_df['continent'] == 'Europe'].copy()

# Ensure we're only using localities that match the criteria from Exercise 7a
european_localities = european_localities[
    (european_localities['LONG'] > -25) & (european_localities['LONG'] < 40) &
    (european_localities['LAT'] > 35) &
    (european_localities['MN_Unit'].isin(mn_order_list[2:19]))  # MN2-MQ19
]

# Calculate expected proportion of first occurrences using the logistic regression model
european_localities['expected_proportion'] = model.predict_proba(
    european_localities['preceding_square_total_occurrences'].values.reshape(-1, 1)# Filter for European localities
european_localities = locality_df[locality_df['continent'] == 'Europe'].copy()

# Ensure we're only using localities that match the criteria from Exercise 7a
european_localities = european_localities[
    (european_localities['LONG'] > -25) & (european_localities['LONG'] < 40) &
    (european_localities['LAT'] > 35) &
    (european_localities['MN_Unit'].isin(mn_order_list[2:19]))  # MN2-MQ19
]

# Calculate expected proportion of first occurrences using the logistic regression model
european_localities['expected_proportion'] = model.predict_proba(
    european_localities['preceding_square_total_occurrences'].values.reshape(-1, 1)

Exercise 10. For each time unit, plot localities on a map covering the
coordinates defined in exercise 7a and indicate their significance level with
a sliding color scheme. Highlight localities that have p-value less than
0.05 (i.e. probability of observations is less than 0.05). Describe briefly
the overall patterns that you observe.
